<a href="https://colab.research.google.com/github/SaiTulasi69/Signifyd-Challenge/blob/main/Signifyd_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
# importing pandas library
import pandas as pd

# loading the dataset with a csv file from github
df = pd.read_csv("https://raw.githubusercontent.com/SaiTulasi69/Signifyd-Challenge/main/Customer_Account.csv", header=0)

In [60]:
# a glance at the first 10 elements in the dataframe
df.head(10)

,Date,CustomerAccountID,EventType
0,2015-01-01,joe@signifyd.com,PURCHASE
1,2015-02-01,fraudster@fraud.com,FRAUD_REPORT
2,2015-02-03,fraudster@fraud.com,FRAUD_REPORT
3,2015-02-10,joe@signifyd.com,PURCHASE
4,2015-02-14,fraudster@fraud.com,PURCHASE
5,2015-03-15,joe@signifyd.com,PURCHASE
6,2015-05-01,joe@signifyd.com,PURCHASE
7,2015-10-01,joe@signifyd.com,PURCHASE


In [61]:
# changing the EventType column's datatype to category
df["EventType"] = df["EventType"].astype("category")

In [62]:
# checking the non-null values and datatypes of all columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Date               8 non-null      object  
 1   CustomerAccountID  8 non-null      object  
 2   EventType          8 non-null      category
dtypes: category(1), object(2)
memory usage: 388.0+ bytes


In [63]:
# importing the datetime module
from datetime import timedelta, date, datetime

# defining methods to classify transactions based on the user's history
class Event:
  # defining a constructor
    def __init__(self, customerAccount):
        self.customerAccountID = customerAccount
        self.historyAllTrans = list()
        self.historyPurchase = list()
        self.eventype = list()
        self.countDictionary = {"PURCHASE":0, "FRAUD_REPORT":0}

  # defining a method to determine the status of the transaction
    def getStatus(self):
        if (len(self.historyAllTrans) == 1) and len(self.historyPurchase) == 1:
          print(str(datetime.strptime(self.historyPurchase[-1], "%Y-%m-%d").date()) + "," + self.customerAccountID + ",NO_HISTORY")
        elif (self.countDictionary["FRAUD_REPORT"] == 0):
          unconfirmed = 0
          goodhistory = 0
          todays_trans = datetime.strptime(self.historyPurchase[-1], "%Y-%m-%d")
          for i in range(len(self.historyPurchase)-1):
            dateInArray = datetime.strptime(self.historyPurchase[i], "%Y-%m-%d")
            diff = todays_trans - dateInArray
            if diff.days > 90:
              goodhistory += 1
            else:
              unconfirmed += 1
          if goodhistory > 0:
            print(str(datetime.strptime(self.historyPurchase[-1], "%Y-%m-%d").date()) + "," + self.customerAccountID + ",GOOD_HISTORY:" + str(goodhistory))
          else:
            print(str(datetime.strptime(self.historyPurchase[-1], "%Y-%m-%d").date()) + "," + self.customerAccountID + ",UNCONFIRMED_HISTORY:" + str(unconfirmed))
        elif (self.countDictionary["FRAUD_REPORT"] >= 1 and len(self.eventype) != 0):
          print(str(datetime.strptime(self.historyPurchase[-1], "%Y-%m-%d").date()) + "," + self.customerAccountID + ",FRAUD_HISTORY:" + str(self.countDictionary["FRAUD_REPORT"]))

  # adding the details of a transaction and keeping a track of the count of each transaction type        
    def addDetails(self, date, eventype):
      self.historyAllTrans.append(date)
      if eventype == "PURCHASE":
        self.historyPurchase.append(date)
        self.eventype.append(eventype)
      self.countDictionary[eventype] += 1
        

In [64]:
# defining a dictionary for all customer details where the user id is the key 
customer_dictionary = dict()
# iterating through all records and adding the customer details in dictionary if it doesn't exist
for index, row in df.iterrows():
  if row['CustomerAccountID'] not in customer_dictionary:
    customer_dictionary[row['CustomerAccountID']] = Event(row['CustomerAccountID'])
  customer_dictionary[row['CustomerAccountID']].addDetails(row['Date'],row['EventType'])
  customer_dictionary[row['CustomerAccountID']].getStatus()

2015-01-01,joe@signifyd.com,NO_HISTORY
2015-02-10,joe@signifyd.com,UNCONFIRMED_HISTORY:1
2015-02-14,fraudster@fraud.com,FRAUD_HISTORY:2
2015-03-15,joe@signifyd.com,UNCONFIRMED_HISTORY:2
2015-05-01,joe@signifyd.com,GOOD_HISTORY:1
2015-10-01,joe@signifyd.com,GOOD_HISTORY:4
